In [1]:
from tqdm import tqdm

import pandas as pd
import cv2
from PIL import Image
import numpy as np

from pipeline.clip_wrapper import ClipWrapper, MODEL_DIM
from pipeline.download_videos import VIDEO_DIR, REPO_ROOT, DATA_DIR

FRAME_EXTRACT_RATE_SECONDS = 5  # Extract a frame every 5 seconds
IMAGES_DIR = DATA_DIR / "images"

DATAFRAME_PATH = DATA_DIR / "dataset.parquet"

/Users/sidneyradcliffe/miniforge3/envs/semvideo-hackathon-230523/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clip_wrapper = ClipWrapper()

In [3]:
def get_clip_vectors(video_path):
    cap = cv2.VideoCapture(str(video_path))
    num_video_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    extract_every_n_frames = FRAME_EXTRACT_RATE_SECONDS * fps
    for frame_idx in tqdm(range(num_video_frames), desc="Running CLIP on video"):
        ret, frame = cap.read()
        if frame_idx % extract_every_n_frames != 0:
            continue
        image = Image.fromarray(frame[..., ::-1])
        clip_vector = clip_wrapper.images2vec([image]).squeeze().numpy()
        timestamp_secs = frame_idx / fps
        yield clip_vector, image, timestamp_secs, frame_idx
    cap.release()

In [4]:
results = []
for i, video_path in enumerate(
    tqdm(list(VIDEO_DIR.glob("*.mp4")), desc="Processing videos")
):
    video_id = video_path.stem
    extracted_images_dir = IMAGES_DIR / video_id
    extracted_images_dir.mkdir(exist_ok=True, parents=True)
    for clip_vector, image, timestamp_secs, frame_idx in get_clip_vectors(video_path):
        image_path = extracted_images_dir / f"{frame_idx}.jpg"
        image.save(image_path)
        results.append(
            [
                video_id,
                frame_idx,
                timestamp_secs,
                str(image_path.relative_to(REPO_ROOT)),
                *clip_vector,
            ]
        )
df = pd.DataFrame(
    results,
    columns=["video_id", "frame_idx", "timestamp", "image_path"]
    + [f"dim_{i}" for i in range(MODEL_DIM)],
)
print(f"Saving data to {DATAFRAME_PATH}")
df.to_parquet(DATAFRAME_PATH, index=False)

Processing videos: 100%|██████████| 5/5 [00:16<00:00,  3.30s/it]

Saving data to /Users/sidneyradcliffe/repos/semvideo-hackathon-230523/data/dataset.parquet
